In [ ]:

#### Don't RUN, need to add deleteting md5 fields
#### Change the frames to raw in the get methods and put it to tibana


from core.utils import Tibanna
from core import ff_utils
import time

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def Sets_from_Pub(pub_list):
    set_list = []
    for pub in pub_list:
        sets = ff_utils.get_metadata(pub, connection=ff).get('exp_sets_prod_in_pub')
        if sets:
            set_list.extend(sets)
    return set_list
    
def Exps_from_Set(set_list, only_replicates=True):
    """By default filters out the non replicate sets"""
    exp_list = []
    for exp_set in set_list:
        if only_replicates:
            set_info = ff_utils.get_metadata(exp_set, connection=ff)
            if set_info['experimentset_type'] == 'replicate':
                exps = set_info.get('experiments_in_set')
                if exps:
                    exp_list.extend(exps)
        else:
            exps = ff_utils.get_metadata(exp_set, connection=ff).get('experiments_in_set')
            if exps:
                exp_list.extend(exps)
    return exp_list

def Files_from_Exp(exp_list):
    file_list = []
    for exp in exp_list:
        files = ff_utils.get_metadata(exp, connection=ff).get('files')
        if files:
            file_list.extend(files)
    return file_list
            
def GetFiles(source):
    data_type = source[1]
    data_list = source[0]
    if data_type == 'file':
        return data_list
    elif data_type == 'exp':
        return Files_from_Exp(data_list)
    elif data_type == 'exp_set':
        return Files_from_Exp(Exps_from_Set(data_list))
    elif data_type == 'pub':
        return Files_from_Exp(Exps_from_Set(Sets_from_Pub(data_list)))

def RetireFile(file_fastq):
    file_data = ff_utils.get_metadata(file_fastq, connection=ff)
    # skip deleted files
    if file_data['status'] == 'deleted':
        return
    if file_data.get('aliases'):
        new_aliases = []
        for alias in file_data['aliases']:
            add_num = 0
            new_alias = alias + '_deleted'
            # check if the alias is not used, if it is, increment name by number 1
            while ff_utils.get_metadata(new_alias, connection = ff).get('code') != 404:
                add_num += 1
                new_alias = alias + '_deleted_' + str(add_num)
            new_aliases.append(new_alias)
            
            # TODO
            # delete md5 related fields to prevent unique key problems
            #
            #
            #
            #
            #
            
        patch_data = {'status': "deleted", 'aliases': new_aliases}
        e = ff_utils.patch_metadata(patch_data, obj_id=file_data['uuid'] ,connection=ff)
        if e.get('status') != 'success':
            print 'problem with', file_data['uuid']
            print e
            return
        else:
            return True
    else:
        # TODO
        # delete md5 related fields to prevent unique key problems
        #
        #
        #
        #
        #
        patch_data = {'status': "deleted"}
        e = ff_utils.patch_metadata(patch_data, obj_id=file_data['uuid'] ,connection=ff)
        if e.get('status') != 'success':
            print 'problem with', file_data['uuid']
            print e
            return
        else:
            return True
    

In [ ]:
# Examples for different object lists
# You can use publication, experimentset, experiment or file list to select different set of files
# Make a tupple of list and the object type; pub, exp_set, exp, or file
Publications =    (['cf0e49aa-173c-49d1-a7c7-22acbc83c064'], 'pub')
Experiment_sets = (['4DNESEYZC9D9', '4DNESWFRUPLR'], 'exp_set')
Experiments =     (['4DNEXPTUMT22', '4DNEX7TEPATH'], 'exp')
Files =           (['sample_file'], 'file')

# create your object list
my_publications = (['dciclab:pub_rao', 'dcic:sanborn2015', 'dciclab:pub_jin', 'dciclab:pub_selvaraj'], 'pub')

my_files = GetFiles((['dcic:test_file0001'],'file'))


print str(len(my_files)), "files to be deleted"

deleted = 0
for file_fastq in my_files:
    if RetireFile(file_fastq):
        deleted += 1
        
print str(deleted), 'files deleted'
